# Homework 4: Graphical LASSO

## Problem 1: Matrix Completion for Movielens

### Part A: 

Download the Movielens 100k dataset:

https://grouplens.org/datasets/movielens/100k/

This dataset consists of 100k entries of a 1000 by 1700 matrix, so there are many missing entries. The goal is to complete the matrix and recommend movies to a user if the completed entry has a high rating. 

After reading this data into python, naively complete the ratings for each movie by first computing the movie's median rating over known entries, and complete the missing ratings for the movie with this median rating.

Thinking of each user as random vector over the movie ratings, use *sklearn.covariance.GraphicalLassoCV* to fit a precision matrix to the random variables of movie ratings (remember to subtract the means before computing the empirical covariance matrix). 

Use this precision matrix and the conditional mean formula to complete the matrix. Plot the completed matrix as a 1000 by 1700 pixel image using *matplotlib.pyplot*.

### Part B: 

Perform 25 bootstrap runs to estimate the generalization error of this procedure. For each of the 25 runs, randomly choose a training set of 70k known entries to complete the matrix as in Part A, and then compute the $\ell_1$ empirical risk on the remaining 30k known entries of test data. Provide a box and whisker plot for these empirical risks. 

### Part A:

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.covariance import GraphicalLassoCV
import matplotlib.pyplot as plt
import numpy.random as rd
import random
import plotly.express as px
import numpy.linalg as la
from sklearn.linear_model import LogisticRegression

In [ ]:
df=pd.read_csv('u.data',sep='\t', names=['user','item','rating','timestamp'])
df=df.drop(columns=['timestamp'])

In [ ]:
from scipy.sparse import csr_matrix
df_total=csr_matrix((np.array(df['rating']),(np.array(df['user'])-1,np.array(df['item'])-1)),shape=(943,1682)).toarray()

In [ ]:
df_raw=np.array(df_total)

In [ ]:
for i in range(1682):
    tempCol = df_total[:,i]
    reCol = tempCol[tempCol!=0]
    tempCol[tempCol==0]=np.median(reCol)
    df_total[:,i]=tempCol

In [ ]:

df_copy = np.array(df_total).astype(np.float)
means=df_copy.mean(axis=0)
stds=df_copy.std(axis=0)
scaler = preprocessing.StandardScaler()
df_copy = scaler.fit_transform(df_copy)

In [ ]:
rem_col = (df_copy!=0).sum(axis=0) != 0
one_col = (df_copy!=0).sum(axis=0) == 0
df_pro = df_copy[:,rem_col]

In [ ]:
df_raw.shape

In [ ]:

alphaRange = (10.0 ** np.arange(0,3)).tolist()
model=GraphicalLassoCV(alphas=alphaRange, mode='cd')
model.fit(df_pro)

In [ ]:
cov=model.covariance_
K = model.precision_

In [ ]:
df_new = np.array(df_raw)
indices = np.array([i for i in range(1682)])
for i in range(943):
    cur_ind=df_raw[i,rem_col]!=0
    pre_ind=df_raw[i,rem_col]==0
    temp = (means[rem_col])[pre_ind]-cov[np.ix_(pre_ind,pre_ind)]@K[np.ix_(pre_ind,cur_ind)]\
    @(df_pro[np.ix_([i],cur_ind)].flatten())*(stds[rem_col])[pre_ind]
    df_new[i,indices[rem_col][pre_ind]]=np.round(temp)

for i in indices[one_col]:
    df_new[:,i]=int(round(means[i]))

In [ ]:

plt.imshow(df_new)

### Part B

In [ ]:

tol_ind = np.array([i for i in range(100000)])
errors = np.array([0]*25)


for j in range(25):
    print('step'+str(j+1))
    train_ind=random.sample(range(100000),70000)
    test_ind= np.setdiff1d(tol_ind,train_ind)
    train_cur=df.loc[index_cur,:]
    test_cur=df.loc[test_ind,:]


    temp_df_total=csr_matrix((np.array(train_cur['rating']),\
                              (np.array(train_cur['user'])-1,np.array(train_cur['item'])-1)),shape=(943,1682)).toarray()
    temp_df_raw =np.array(temp_df_total)

    for i in range(1682):
        tempCol = temp_df_total[:,i]
        reCol = tempCol[tempCol!=0]
        if len(reCol)>0:
            tempCol[tempCol==0]=np.median(reCol)
        else:
            print('No data in Column'+str(i+1))
            tempCol=np.array([3]*943)
        temp_df_total[:,i]=tempCol

    temp_df_copy = np.array(temp_df_total).astype(np.float)
    temp_means=temp_df_copy.mean(axis=0)
    temp_stds=temp_df_copy.std(axis=0)
    temp_scaler = preprocessing.StandardScaler()
    temp_df_copy = temp_scaler.fit_transform(temp_df_copy)

    temp_rem_col = (temp_df_copy!=0).sum(axis=0) != 0
    temp_one_col = (temp_df_copy!=0).sum(axis=0) == 0
    temp_df_pro = temp_df_copy[:,temp_rem_col]

    temp_alphaRange = (10.0 ** np.arange(0,3)).tolist()
    temp_model=GraphicalLassoCV(alphas=temp_alphaRange, mode='cd')
    temp_model.fit(temp_df_pro)

    temp_cov = temp_model.covariance_
    temp_K = temp_model.precision_

    temp_df_new = np.array(temp_df_raw)
    temp_indices = np.array([i for i in range(1682)])
    for i in range(943):
        temp_cur_ind=temp_df_raw[i,temp_rem_col]!=0
        temp_pre_ind=temp_df_raw[i,temp_rem_col]==0
        temp = (temp_means[temp_rem_col])[temp_pre_ind]-temp_cov[np.ix_(temp_pre_ind,temp_pre_ind)]\
        @ temp_K[np.ix_(temp_pre_ind,temp_cur_ind)]\
        @(temp_df_pro[np.ix_([i],temp_cur_ind)].flatten())*(temp_stds[temp_rem_col])[temp_pre_ind]
        temp_df_new[i,temp_indices[temp_rem_col][temp_pre_ind]]=np.round(temp)

    for i in temp_indices[temp_one_col]:
        temp_df_new[:,i]=int(round(temp_means[i]))

    error = 0
    for i in range(test_cur.shape[0]):
        error += np.abs(temp_df_new[test_cur.iloc[i,0]-1,test_cur.iloc[i,1]-1]-test_cur.iloc[i,2])
    errors[j]=error

In [ ]:

px.box(y=errors,title='L1 Empirical Risk')

## Problem 2: Graphical LASSO fit for Riboflavin Data

Append the target ($y$) variable to the Riboflavin feature dataset ($X$), remove the empirical means from the columns of $(y\:X)$, and then use *sklearn.covariance.GraphicalLassoCV* to fit a precision matrix for this appended dataset. Note that this method automatically chooses the $\lambda$ parameter using cross validation.

Under the assumption of normality, use precision matrix on the dataset plus target to approximate the target as a sparse linear combination of the the predictors (as discussed in Lecture 08). Sort all of the target values $y$ in increasing order, and plot these against the predicted values to visually assess the regression. Describe your results and compare with the LASSO solution from HW3 Problem 2. 


In [ ]:
df_rx=pd.read_csv('riboflavin/riboflavinx.csv')
df_ry=pd.read_csv('riboflavin/riboflaviny.csv')
rx=np.array(df_rx.iloc[:,1:4089])
ry=np.array(df_ry.iloc[:,1])
ry = np.expand_dims(ry, axis=0)
selected = random.sample([i for i in range(4089)],200)
rx = rx[:,selected]

In [ ]:
ribof=np.concatenate((rx,ry.T),axis=1)
means = ribof.mean(axis=0)
ribof-=means

In [ ]:
model2=GraphicalLassoCV(mode='cd',n_jobs=-1)
model2.fit(ribof)

In [ ]:
cov_ribof = model2.covariance_
K_ribof = model2.precision_

In [ ]:
y_pred = np.array([0]*71)
for i in range(71):
    y_pred[i] = means[200]-cov_ribof[200,200]*K_ribof[200,0:200]@ribof[i,0:200]

In [ ]:
px.scatter(x=y_pred, y=ribof[:,200])

In [ ]:
0.5*la.norm(y_pred-ribof[:,200])**2/71

In [ ]:
def soft_thres(x,lamb):
    if(x>lamb):
        return(x-lamb)
    if(x<-lamb):
        return(x+lamb)
    return 0
def coord_des_lasso(y, X, beta0, lamb, error, maxstep):
    n=X.shape[0]
    p=X.shape[1]
    beta = beta0
    r_prev = 0.5*la.norm(y-X@beta)**2+lamb*la.norm(beta,ord=1)
    step = 0
    while(step < maxstep):
        #print("#######")
        #print("Step"+str(step))
        for i in range(p):
            beta_temp=np.array(beta)
            beta_temp[i]=0
            y_temp = y-X@beta_temp
            beta[i]=soft_thres(np.dot(y_temp,X[:,i]),lamb)/(la.norm(X[:,i])**2)
        r_cur = 0.5*la.norm(y-X@beta)**2+lamb*la.norm(beta,ord=1)
        if(np.abs(r_cur-r_prev)<error or step==maxstep-1):
            err_final=np.abs(r_cur-r_prev)
            break
        else:
            r_prev=r_cur
            step=step+1
    return beta, step, err_final

In [ ]:
prederr_record=np.zeros((5,25))
ry=ry.flatten()
ind = [-2,-1,0,1,2]
for i in range(5):
    print('#####')
    print("lambda_"+str(i+1))
    for j in range(25):
        print("  step_"+str(j+1))
        train_index=rd.choice(np.arange(71),50,replace=False)
        test_index=np.setdiff1d(np.arange(71),train_index)

        train_X = rx[train_index,:]
        train_y = ry[train_index]
        test_X = rx[test_index,:]
        test_y = ry[test_index]

        beta0=np.zeros(train_X.shape[1])
        lamb = np.log(train_X.shape[1])*(10**ind[i])
        beta, step, err = coord_des_lasso(train_y,train_X,beta0,lamb,0.1,100)

        pred_err = 0.5*la.norm(test_y-test_X@beta)**2
        prederr_record[i,j]=pred_err
prederr_record /= 21

In [ ]:
df_temp = pd.DataFrame(prederr_record.T, columns=['k=-2','k=-1','k=0','k=1','k=2'])
df_tidy = pd.melt(df_temp)

px.box(df_tidy, x='variable', y='value', color='variable',title='Predictive Errors Using Different Lambdas')

### Comment
While we can achieve good results through Graphical Lasso, it takes a lot of time. Thus, we have to select a subset of all the features and then do the graphical lasso. But the loss of information leads to a bad performance.

## Problem 3: Graphical LASSO and LDA for Prostate Dataset


In this problem, you will modify the code from HW3 Problem 3 to train a regularized Linear Discriminant Analysis (LDA) classifier. Split the data as in HW3 Problem 3, and use the training data to fit conditional means and conditional precision matrices to the two classes. That is, let $C_0=\{i:y_i=0\}$ and $C_1=\{i:y_i=1\}$
$$
\hat\mu_0 = \frac{1}{\text{card} C_0} \sum_{i\in C_0} x_i\text{ and }\hat\mu_1 = \frac{1}{\text{card} C_1} \sum_{i\in C_1} x_i
$$
and
$$
\hat\Sigma_0 = \frac{1}{\text{card} C_0} \sum_{i\in C_0} (x_i-\hat\mu_i)^T(x_i-\hat\mu_i)\text{ and }\hat\Sigma_1 = \frac{1}{\text{card} C_1} \sum_{i\in C_1} (x_i-\hat\mu_i)^T(x_i-\hat\mu_i).
$$
Use *sklearn.covariance.GraphicalLassoCV* to fit $K_0$ and $K_1$ using $\Sigma_0$ and $\Sigma_1$.

We define the Linear Discriminant as
$$
f(x) = (x-\hat\mu_0)^T K_0(x-\hat\mu_0) - \ln\det K_0 - (x-\hat\mu_1)^T K_1(x-\hat\mu_1) + \ln\det K_1
$$
which induces the prediction
$$
\text{pred}_\eta(x) = \left\{\begin{array}{cl} 0 &f(x)\leq \eta\\ 1 & f(x)>\eta\end{array}\right.
$$
Here, $\eta$ is chosen to manage the tradeoff between Type I and Type II errors. 

### Part A: 

In LDA, it is often assumed that $K_0=K_1$ so that the linear discriminant function simplifies. This is called *heteroscedasticity*. Does this assumption appear to be true for the prostate data set?

### Part B:

Run your code one time, and display the ROC (receiver operating characteristic) curve for the LDA classifier over the test dataset. A good measure of the usefulness of a classifier is the area under the ROC curve, and a classifier is good if the area under the curve is very close to $1$. It the LDA classifier "good" according to this measure?

### Part C: 

The logistic regression classifier from HW3 Problem 3 can also be modified to admit an ROC curve. After fitting $p(y=0|x)$ and $p(y=1|x)$, one can predict $y=1$ for $x$ if

$$
\ln p(y=1|x) - \ln p(y=0|x) > \eta.
$$

For a single run of training/testing, simultaneously display the ROC curve for this LDA classifier and the $\ell_1$ regularized logistic regression classifier trained on the same data. Discuss your observations.


### Part D: 

Discuss reasons why you would prefer the regularized logisitic regression classifier over this regularized LDA classifier and vice versa.


### Part A

In [ ]:
pX = pd.read_csv('prostate/prostate.csv')
pY = pX.iloc[:,-1]
pX = pX.iloc[:,1:6034]
selected = random.sample([i for i in range(4089)],200)
pX = pX.iloc[:,selected]

In [ ]:
indices = np.array([i for i in range(102)])
pos_ind = indices[pY==1]
neg_ind = indices[pY==0]

pos_X = pX.iloc[pos_ind,:]
neg_X = pX.iloc[neg_ind,:]

In [ ]:
pos_means = pos_X.mean(axis=0)
neg_means = neg_X.mean(axis=0)
pos_X -= pos_means
neg_X -= neg_means

In [ ]:
model3=GraphicalLassoCV(n_jobs=-1)
model3.fit(pos_X)

In [ ]:
model4=GraphicalLassoCV(n_jobs=-1)
model4.fit(neg_X)

In [ ]:
K1 = model3.precision_
K2 = model4.precision_

In [ ]:
(K1!=K2).sum()

### Comment
This is not true for this random subset of the Prostate dataset.

### Part B

In [ ]:
def LDA(x, yeta):
    judge = (x-pos_means)@K1@(x-pos_means).T-np.log(la.det(K1))-(x-neg_means)@K2@(x-neg_means).T+np.log(la.det(K2))-yeta
    if judge > 0:
        return 1
    else:
        return 0
def ROC_compute(yeta):
    TP=0
    TN=0
    FP=0
    FN=0
    for i in range(71):
        x = pX.iloc[i,:]
        temp_pred=LDA(x,yeta)
        if temp_pred==1:
            if pY[i]==1:
                TP+=1
            else:
                FP+=1
        else:
            if pY[i]==1:
                FN+=1
            else:
                TN+=1
    return TP/(TP+FN), FP/(FP+TN)

In [ ]:
def crit(x, yeta):
    return (x-pos_means)@K1@(x-pos_means).T-np.log(la.det(K1))-(x-neg_means)@K2@(x-neg_means).T+np.log(la.det(K2))-yeta
#pX.apply(crit,axis=1,args=(0))
crit_result=pX.apply(crit,axis=1,args=(0,))
min_yeta= np.min(crit_result)-0.1
max_yeta= np.max(crit_result)+0.1

In [ ]:
num_of_yeta = 40
yetas = np.arange(min_yeta, max_yeta, (max_yeta-min_yeta)/num_of_yeta)

In [ ]:
TPR = np.zeros(num_of_yeta)
FPR = np.zeros(num_of_yeta)
for i in range(num_of_yeta):
    yeta = yetas[i]
    TPR[i], FPR[i] = ROC_compute(yeta)

In [ ]:
px.line(x=TPR,y=FPR,title='ROC Curve of Subset Graphical Lasso')

### Comment
The 'subset' graphical lasso seems not to be good due to the loss of information.

### Part C

In [ ]:
train_index = rd.choice(np.arange(101),80,replace=False)
test_index = np.setdiff1d(np.arange(101),train_index)
pX_train = pX.iloc[train_index,:]
pY_train = pY[train_index]
pX_test = pX.iloc[test_index,:]
pY_test = np.array(pY[test_index])

In [ ]:
model5 = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=37).fit(pX_train, pY_train)

In [ ]:
log_prob = model5.predict_log_proba(pX_test)
log_res = (log_prob[:,1]-log_prob[:,0])

In [ ]:
min_yeta2 = np.min(log_res)
max_yeta2 = np.max(log_res)

num_of_yeta = 40
yetas2 = np.arange(min_yeta2, max_yeta2, (max_yeta2-min_yeta2)/num_of_yeta)

In [ ]:
def ROC_compute_2(yeta):
    y_pred = np.where(log_res>yeta,1,0)
    TP=0
    TN=0
    FP=0
    FN=0
    for i in range(len(y_pred)):
        if y_pred[i]==1:
            if pY_test[i]==1:
                TP+=1
            else:
                FP+=1
        else:
            if pY_test[i]==1:
                FN+=1
            else:
                TN+=1
    return TP/(TP+FN), FP/(FP+TN)

In [ ]:
TPR2 = np.zeros(num_of_yeta)
FPR2 = np.zeros(num_of_yeta)
for i in range(num_of_yeta):
    yeta2 = yetas2[i]
    TPR2[i], FPR2[i] = ROC_compute_2(yeta2)

In [ ]:
px.line(x=TPR2,y=FPR2,title='ROC Curve of L1 Penalized Logistic Regression')

### Comment
The performance of Logistic Regression in this case in  metric of ROC is rather bad.

### Part D
As we can see, if we judge the performance in the metric of ROC curve, the regularized LDA is better than the regularized logisitic regression.

## Problem 4: S&P 500 Log Returns Data


Consider the following code for the the conditional dependence structure on the log-returns of 404 symbols in the S&P 500 data across two successive days.

### Part A: Describe the plots produced by this code. First, explain what the plots are displaying, and then interpret what the plots tell you about the the data.

### Part B: Modify this code to produce a sparse precision matrix for the log returns of the symbols based on the first 150 days of information only, and use the structure of this precision matrix to determine small subsets of predictors from the first day and responses from the second day that exhibit a dependence structure. Replace each response variable from this small list with an indicator variable that is 1 if the variable is positive and 0 if the variable is negative, and perform logistic regression of the small subset of predictors on this indicator. Plot the ROC curves for each of these logisitic regression models on the next 100 days of data, and discuss your results.

### Part C: For each response symbol found in Part B, perform $\ell^1$-penalized logisitic regression using all 404 symbols log-returns on the previous day. Use the function *sklearn.linear_model.LogisiticRegression* with the argument *penalty='l1'* to train on the first 150 days of data, and then provide ROC curves for these models on the next 100 days of data. Compare these results with the results from Part B. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("./finance/data.csv")

# Symbols with nan data
nanix = [7, 52, 56, 65, 70, 71, 72, 75, 94, 99, 122, 125, 138, 170, 174, 177, 193, 203, 226, 260, 282, 308, 309, 326, 335, 352, 365, 389, 404]

data = np.zeros((433-len(nanix),251))
k = 0

for i in range(433):
    q = np.diff(np.log(np.array(df['Open.' + str(i+1)][2:].astype(float))))
    if np.sum(np.isnan(q)) == 0:
        data[k,:]=np.diff(np.log(np.array(df['Open.' + str(i+1)][2:].astype(float))))
        k=k+1
        
scaler = StandardScaler()
scaler.fit(data)
x = scaler.transform(data)

# Data from successive trading days
seq_data = np.concatenate((x[:,0:250], x[:,1:251]), axis=0)
   
sig = np.ma.cov(seq_data) #Removes NANs from covariance computation

In [ ]:
from sklearn.covariance import graphical_lasso
import plotly.graph_objects as go
import networkx as nx

def display_graph(A, node_color, node_text):
    G = nx.from_numpy_matrix(A)
    X = nx.spring_layout(G)
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0 = X[edge[0]]
        x1 = X[edge[1]]
        edge_x.append(x0[0])
        edge_x.append(x1[0])
        edge_x.append(None)
        edge_y.append(x0[1])
        edge_y.append(x1[1])
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    for node in G.nodes():
        x = X[node]
        node_x.append(x[0])
        node_y.append(x[1])

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                #title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_trace.marker.color = node_color
    node_trace.text = node_text


    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='2D Network Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="", #text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()
    
def display_graph_3D(A, node_color, node_text):
    G = nx.from_numpy_matrix(A)
    X = nx.spring_layout(G, dim=3)
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        x0 = X[edge[0]]
        x1 = X[edge[1]]
        edge_x.append(x0[0])
        edge_x.append(x1[0])
        edge_x.append(None)
        edge_y.append(x0[1])
        edge_y.append(x1[1])
        edge_y.append(None)
        edge_z.append(x0[2])
        edge_z.append(x1[2])
        edge_z.append(None)

    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_z = []
    for node in G.nodes():
        x = X[node]
        node_x.append(x[0])
        node_y.append(x[1])
        node_z.append(x[2])

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=5,
            colorbar=dict(
                thickness=15,
                #title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_trace.marker.color = node_color
    node_trace.text = node_text


    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title='3D Network Graph',
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="", #text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()
    
A = np.matrix([[0, 1, 0, 1],
               [1, 0, 1, 1],
               [0, 1, 0, 1],
               [1, 0, 1, 0]])

node_color = [2, 3, 2, 3]
node_text = ['1', '2', '3', '4']


display_graph_3D(A, node_color, node_text)




In [ ]:
emp_cov = sig.data
cov, pre = graphical_lasso(emp_cov,0.6,verbose=True)

tau = np.percentile(np.abs(np.ndarray.flatten(pre)),99)
fig = go.Figure(data=go.Heatmap(z=pre, x=[], y=[], zmin=-tau, zmax=tau))
fig.show()

In [ ]:
A = np.abs(pre - np.diag(np.diag(pre)))
#tau = np.percentile(np.abs(A),99)
#A = (A > tau)  

A = (A > 0)

fig = go.Figure(data=go.Histogram(x=np.matrix.flatten(np.abs(pre[A]))))
fig.show()

A = A.astype(int)
fig = go.Figure(data=go.Heatmap(z=A, x=[], y=[]))
fig.show()

node_color = np.diag(pre)
node_text = [] * 404

display_graph(A, node_color, node_text)

display_graph_3D(A, node_color, node_text)

### Part A
**(1)** The heatmap of the precision matrix is shown. We can see the diagonal elements are the significant ones while other elements are minor. That indicates small correlation between variables.

**(2)** The histogram of non-zero elements that are not in the diagonal line is shown. We can see most of them are relatively small.

**(3)** The heatmap of all non-zero elements that are not in the diagonal line is shown. By converting all those elements into values of 1, we can clearly catch those non-zero elements.

**(4)** The 2D network graph of all variables in the design matrix is shown. We can see that most nodes are not correlated.

**(5)** The 3D network graph of all variables in the design matrix is shown. Also we can see that most nodes are not correlated.

### Part B

In [ ]:
des_data = seq_data[:,0:150]

In [ ]:
sig_des = np.ma.cov(des_data)
emp_cov = sig_des.data
cov, pre = graphical_lasso(emp_cov,0.6,verbose=True)


In [ ]:
A = np.abs(pre - np.diag(np.diag(pre)))
A = (A > 0).astype(int)

In [ ]:
response_list = np.arange(404)[(A[404:808,0:404].sum(axis=1)!=0).flatten()]

In [ ]:
def ROC_compute_3(yeta):
    y_pred = np.where(temp_log_res>yeta,1,0)
    TP=0
    TN=0
    FP=0
    FN=0
    for i in range(len(y_pred)):
        if y_pred[i]==1:
            if temp_y[i]==1:
                TP+=1
            else:
                FP+=1
        else:
            if temp_y[i]==1:
                FN+=1
            else:
                TN+=1
    return TP/(TP+FN), FP/(FP+TN)

In [ ]:
response_list

In [ ]:
for i in range(len(response_list)):
    index = response_list[i]
    predictor_list = np.arange(404)[(A[404+index,:]==1)[0:404]]
    temp_X = des_data[predictor_list,:].T
    temp_y = np.where(des_data[404+index,:]>0,1,0)
    temp_model = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=37).fit(temp_X, temp_y)
    
    temp_X_test = seq_data[predictor_list,150:250].T
    temp_y_test = np.where(seq_data[404+index,150:250]>0,1,0)

    temp_log_prob = temp_model.predict_log_proba(temp_X_test)
    temp_log_res = (temp_log_prob[:,1]-temp_log_prob[:,0])

    temp_min_yeta2 = np.min(temp_log_res)-0.1
    temp_max_yeta2 = np.max(temp_log_res)+0.1

    num_of_yeta = 40
    temp_yetas2 = np.arange(temp_min_yeta2, temp_max_yeta2, (temp_max_yeta2-temp_min_yeta2)/num_of_yeta)
   
    temp_TPR2 = np.zeros(num_of_yeta)
    temp_FPR2 = np.zeros(num_of_yeta)
    for j in range(num_of_yeta):
        yeta2 = temp_yetas2[j]
        temp_TPR2[j], temp_FPR2[j] = ROC_compute_3(yeta2)

    px.line(x=temp_TPR2,y=temp_FPR2,title=str(i)+'.'+' ROC Curve of Variable '+str(index)).show()

### Comment
For most symbols, we have an ROC curve close to the diagonal line, which indicates we can gain a model with a pair of TPR and FPR approximately being (0.5, 0.5) (Or a little bit better than that). So we are not gaining a model with high accuracy and precision, but at least we are not gaining a model too bad (They're not below the diagonal line too far). So that's the common performance in the time-series analysis. If we use the similar method with the previous log return variables more than one day, the performance can be improved.

### Part C

In [ ]:
for i in range(len(response_list)):
    temp_X = des_data[0:404,:].T
    temp_y = np.where(des_data[404+index,:]>0,1,0)
    temp_model = LogisticRegression(penalty='l1', solver='saga', max_iter=1000, random_state=37).fit(temp_X, temp_y)
    
    temp_X_test = seq_data[0:404,150:250].T
    temp_y_test = np.where(seq_data[404+index,150:250]>0,1,0)

    temp_log_prob = temp_model.predict_log_proba(temp_X_test)
    temp_log_res = (temp_log_prob[:,1]-temp_log_prob[:,0])

    temp_min_yeta2 = np.min(temp_log_res)-0.1
    temp_max_yeta2 = np.max(temp_log_res)+0.1

    num_of_yeta = 40
    temp_yetas2 = np.arange(temp_min_yeta2, temp_max_yeta2, (temp_max_yeta2-temp_min_yeta2)/num_of_yeta)
   
    temp_TPR2 = np.zeros(num_of_yeta)
    temp_FPR2 = np.zeros(num_of_yeta)
    for j in range(num_of_yeta):
        yeta2 = temp_yetas2[j]
        temp_TPR2[j], temp_FPR2[j] = ROC_compute_3(yeta2)

    px.line(x=temp_TPR2,y=temp_FPR2,title=str(i)+'.'+' ROC Curve of Variable '+str(index)).show()

### Comment
As we can see, the performances of the models using all predictors are similar to the corresponding models using only the response-correlated predictors. That indicates our graphical lasso method is making sense. In conclusion, we can gain models with simple structures and little information loss.

## Problem 5: Choose your own adventure

Using your own dataset (the high-dimensional dataset from Homework 1), perform penalized regression or penalized logistic regression and discuss your results. 

In [ ]:
df1=pd.read_csv('data.csv')
df2=pd.read_csv('labels.csv')
df1=df1.iloc[:,1:20532]
df2=df2.iloc[:,1]

'data.csv' collects the gene expressions. 'labels.csv' represents the kinds of cancer.  
To carry out a logistic regression, we only consider whether the label is 'LUAD' or not, making it a binary-classification problem.

In [ ]:
labels=np.zeros(801)
labels[df2=='LUAD']=1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df1, labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn.svm import l1_min_c
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

cs = l1_min_c(X_train, y_train, loss="log") * np.logspace(0, 7, 16)

clf = linear_model.LogisticRegression(
    penalty="l1",
    solver="liblinear",
    tol=1e-6,
    max_iter=int(1e6),
    warm_start=True,
    intercept_scaling=10000.0,
)

coefs_ = []
error=[]
for c in cs:
    clf.set_params(C=c)
    clf.fit(X_train, y_train)
    coefs_.append(clf.coef_.ravel().copy())
    error.append(np.mean(clf.predict(X_test)-y_test)**2)

In [ ]:
error

In [ ]:
norms=[]
for array in coefs_:
    norms.append(np.sum(array!=0))
norms

In [ ]:
from sklearn.linear_model import LogisticRegression
temp_model = LogisticRegression(fit_intercept=False,solver='liblinear').fit(X_train, y_train) 
np.mean((temp_model.predict(X_test)-y_test)**2)

In [ ]:
np.sum(temp_model.coef_.ravel()!=0)